# 0. Experimental 

In [ ]:
%pip install --upgrade langchain_experimental

In [ ]:
model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke("what is the weather in Boston?")

In [ ]:
from langchain_core.tools import tool
from typing import Annotated

In [ ]:
from scraping.db.documents import ArticleDocument

In [ ]:
@tool
def get_medium_article(link: Annotated[str, "The link of the Medium article"],) -> str:
    """Get a medium article from the given link"""
    filter_options = {"link": link}
    article = ArticleDocument.get(**filter_options)
    return article.content['Content']

In [ ]:
link = "https://medium.com/ai-in-plain-english/claude-3-5-sonnet-why-it-is-better-than-chatgpt-7709d7cbc237"

In [ ]:

r = get_medium_article(link)

In [ ]:
print(r)

In [ ]:
get_medium_article.args_schema.schema()


In [ ]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel

class AnswerWithJustification(BaseModel):
    """An answer to the user question along with justification for the answer."""
    answer: str
    justification: str
    
model = OllamaFunctions(model="llama3-groq-tool-use")
structured_llm = model.with_structured_output(AnswerWithJustification, include_raw=True)
#model = OllamaFunctions(model="llama3-groq-tool-use", format="json")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Use the tools at your disposal at the best of your ability.",
        ),
        ("human", "{input}"),
    ]
)

In [ ]:
tools = [get_medium_article]

In [ ]:
llm = model.bind_tools(tools)

In [ ]:
chain = prompt | llm

In [ ]:
input = "Can you tell me what this Medium article is about ? Here is the link to the article: https://medium.com/ai-in-plain-english/claude-3-5-sonnet-why-it-is-better-than-chatgpt-7709d7cbc237"

In [ ]:
ai_msg = chain.invoke(
    {
        "input": input,
    }
)

In [ ]:
available_tools = {"get_medium_article" : get_medium_article}

In [ ]:
messages = [ai_msg]

In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = available_tools[tool_call["name"].lower()]
    if selected_tool:
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)
    else:
        raise ValueError(f"No tool called {tool_call['name'].lower()} was found")

In [ ]:
messages

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the get_medium_article tool to retrieve the content of the Medium article provided to you.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


In [ ]:
agent = create_tool_calling_agent(model, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
result = agent_executor.invoke({"input": input})

# 1. Install dependencies

In [ ]:
%pip install --upgrade pip
%pip install --upgrade langchain-ollama
%pip install --upgrade langgraph

# 2. Initialize the model

In [ ]:
from langchain_ollama import ChatOllama

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
model = ChatOllama(model="llama3-groq-tool-use", temperature=1)
#model = ChatOllama(model="llama3.1", temperature=0)

In [ ]:
# Create the primary assistant prompt template
from langchain_core.prompts import ChatPromptTemplate
primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant tasked with answering user questions. "
            "You have access to two tools: retrieve_documents and web_search. "
            "For any user questions about LLM agents, use the retrieve_documents tool to get information for a vectorstore. "
            "For any other questions, such as questions about current events, use the web_search tool to get information from the web. ",
        ),
        ("placeholder", "{messages}"),
    ]
)

# 3. Initialize the tools

In [ ]:
from langgraph.prebuilt import ToolNode

@tool
def search(query: str):
    """Run web search on the question.
    
    Args:
        query (str): The query to search for.

    Returns:
        str: The answer to the question.
    """
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."


tools = [search]

tool_node = ToolNode(tools)

# 4. Initialize graph with state

In [ ]:
from langgraph.graph import END, START, StateGraph, MessagesState

In [ ]:
from typing import Annotated, Literal, TypedDict

# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# 5. Define graph nodes

## 5.1 CREATE AN ASSISTANT CLASS

In [ ]:
from langchain_core.runnables import Runnable, RunnableConfig
from typing_extensions import TypedDict
from langchain_core.runnables import Runnable, RunnableConfig
from langgraph.graph.message import AnyMessage, add_messages
from typing import Annotated, List
from langchain_core.prompts import ChatPromptTemplate

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]



class Assistant:
    def __init__(self, runnable: Runnable):
        """
        Initialize the Assistant with a runnable object.

        Args:
            runnable (Runnable): The runnable instance to invoke.
        """
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        """
        Call method to invoke the LLM and handle its responses.
        Re-prompt the assistant if the response is not a tool call or meaningful text.

        Args:
            state (State): The current state containing messages.
            config (RunnableConfig): The configuration for the runnable.

        Returns:
            dict: The final state containing the updated messages.
        """
        while True:
            result = self.runnable.invoke(state)  # Invoke the LLM
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}


# Create the primary assistant prompt template
primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant tasked with answering user questions. "
            "You have access to one tool: search ."
            "For any questions, such as questions about current events, use the search tool to get real time information from the web. ",
        ),
        ("placeholder", "{messages}"),
    ]
)

# Prompt our LLM and bind tools
assistant_runnable = primary_assistant_prompt | model.bind_tools(tools)

## 5.2 Create the graph

In [ ]:
# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", Assistant(assistant_runnable))
workflow.add_node("tools", tool_node)

In [ ]:
# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

In [ ]:
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

In [ ]:
workflow.add_edge("tools", 'agent')

In [ ]:
checkpointer = MemorySaver()

# 6. Define entry point and graph edges 

# 7. Compile the graph

In [ ]:
app = workflow.compile(checkpointer=checkpointer)

In [ ]:
from IPython.display import Image, display
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="What is the weather in San Francisco ? Use the search function to find the latest information.")]},
    config={"configurable": {"thread_id": 1}}
)
final_state["messages"][-1].content# 8. Run Graph


In [ ]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="What is the weather in Paris ? Use the search function to find the latest information.")]},
    config={"configurable": {"thread_id": 1}}
)
final_state["messages"][-1].content# 8. Run Graph